In [2]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [51]:
url_inicial = 'http://www.cya.unam.mx/index.php/cya/issue/archive?issuesPage=4#issues'
url_root = 'http://www.cya.unam.mx/index.php/cya/issue/archive?issuesPage=4#issues'
r=requests.get(url_inicial)

In [52]:
soup = BeautifulSoup(r.text, 'html.parser')

In [53]:
box = soup.find('div', id='issues')
volumen=box.findAll('h4')
vol = [x.find('a').get('href')for x in volumen]
vol

['http://www.cya.unam.mx/index.php/cya/issue/view/52']

In [54]:
vol2 =[]
for i in vol: 
    url_inicial1=i 
    r1 = requests.get(url_inicial1)
    soup1 = BeautifulSoup(r1.text, 'html.parser')
    box1 = soup1.find('div', id='content') 
    volumen1=box1.findAll('table', class_='tocArticle')
    vol1 = [x.find('a').get('href')for x in volumen1]
    vol2+=vol1

In [55]:
vol2

['http://www.cya.unam.mx/index.php/cya/article/view/456',
 'http://www.cya.unam.mx/index.php/cya/article/view/457',
 'http://www.cya.unam.mx/index.php/cya/article/view/458',
 'http://www.cya.unam.mx/index.php/cya/article/view/459',
 'http://www.cya.unam.mx/index.php/cya/article/view/460',
 'http://www.cya.unam.mx/index.php/cya/article/view/461',
 'http://www.cya.unam.mx/index.php/cya/article/view/462']

In [56]:
def get_url_items(sopa,url):
    box1 = soup1.find('div', id='content') 
    volumen1=box1.findAll('table', class_='tocArticle')
    vol1 = [x.find('a').get('href')for x in volumen1]
    return vol2

In [57]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('li.next > a')
    if not next_a or not next_a[0].get('href'):
        break

Estoy en la pagina http://www.cya.unam.mx/index.php/cya/issue/archive?issuesPage=4#issues


In [58]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

7

In [59]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [60]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Contaduría y Administración'
    a='V. Ciencias Sociales'
    tem='Economía, Econometría y Finanzas'
    s_item=BeautifulSoup(r.text,'html.parser')
 #titulo de revista
    try:
        titulo=tire
        content_book['Titulo Revista']=tire
    except AttributeError:
        content_book['Titulo Revista']=None
    #area
    try:
        area=a
        content_book['Area']=a
    except AttributeError:
        content_book['Area']=None
    #tematica
    try:
        tema=tem
        content_book['Tematica']=tem
    except AttributeError:
        content_book['Tematica']=None
    #titulo articulo
    try:
        titu=s_item.find('div', id='articleTitle').get_text(strip=True)
        content_book['Titulo Articulo']=titu
    except AttributeError:
        content_book['Titulo Articulo']=None
    #resumen
    try:
        resu=s_item.find('div', id='articleAbstract').get_text(strip=True)
        content_book['Resumen']=resu.replace('Resumen','')
    except AttributeError:
        content_book['Resumen']=None
    #abstract
    try:
        abst=s_item.find('div', class_='item cover_image').get_text(strip=True)
        content_book['Abstract']=abst
    except AttributeError:
        content_book['Abstract']=None
    #Link incial
    try:
        linkart=url
        content_book['Link Articulo']=linkart
    except AttributeError:
        content_book['Link Articulo']=None
    #link articulo
    try:
        link=s_item.find('a', class_='file').get('href')
        content_book['Link PDF']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link PDF']=None
    return content_book

In [61]:
list_scraper=list_scraper[0:202]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1
estas escrapeando la pag 2
estas escrapeando la pag 3
estas escrapeando la pag 4
estas escrapeando la pag 5
estas escrapeando la pag 6


In [62]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo Revista,Area,Tematica,Titulo Articulo,Resumen,Abstract,Link Articulo,Link PDF
0,Contaduría y Administración,V. Ciencias Sociales,"Economía, Econometría y Finanzas",Editorial Número 213,"Largo, Sinuoso, incluso interrumpido una vez, ...",None,http://www.cya.unam.mx/index.php/cya/article/v...,http://www.cya.unam.mx/index.php/cya/article/v...
1,Contaduría y Administración,V. Ciencias Sociales,"Economía, Econometría y Finanzas",Política ambiental y su impacto en la innovaci...,"NUEVOS CAMPOS DE ESTUDIO, COMO LA ECOLOGÍA IND...",None,http://www.cya.unam.mx/index.php/cya/article/v...,http://www.cya.unam.mx/index.php/cya/article/v...
2,Contaduría y Administración,V. Ciencias Sociales,"Economía, Econometría y Finanzas",Las aproximaciones organizacionales caracteriz...,En este artículo se presenta una visión global...,None,http://www.cya.unam.mx/index.php/cya/article/v...,http://www.cya.unam.mx/index.php/cya/article/v...
3,Contaduría y Administración,V. Ciencias Sociales,"Economía, Econometría y Finanzas",Competencia de las prendas de vestir mexicanas...,EL OBJETIVO DE ESTE ARTÍCULO ES ANALIZAR EL CO...,None,http://www.cya.unam.mx/index.php/cya/article/v...,http://www.cya.unam.mx/index.php/cya/article/v...
4,Contaduría y Administración,V. Ciencias Sociales,"Economía, Econometría y Finanzas",Modelado de la volatilidad y pronóstico del ín...,EN ESTE DOCUMENTO SE EVALÚA LA CONTRIBUCIÓN DE...,None,http://www.cya.unam.mx/index.php/cya/article/v...,http://www.cya.unam.mx/index.php/cya/article/v...
5,Contaduría y Administración,V. Ciencias Sociales,"Economía, Econometría y Finanzas",¿Ciencias duras y ciencias blandas? Una falsa ...,"No comparto, por las razones que expondré más ...",None,http://www.cya.unam.mx/index.php/cya/article/v...,http://www.cya.unam.mx/index.php/cya/article/v...
6,Contaduría y Administración,V. Ciencias Sociales,"Economía, Econometría y Finanzas",Sobre la administración comentarios a un texto...,Con el fin de pensar rigurosamente la cuestión...,None,http://www.cya.unam.mx/index.php/cya/article/v...,http://www.cya.unam.mx/index.php/cya/article/v...


In [63]:
df_catalogo.to_csv('Revista037.04.csv', index=False)

In [ ]:
df=df_catalogo[df_catalogo[0,17,38,59,80,96,107,118,129,140,152,163,174,185,196,207,218,229,240,251,]#.isin(['Editorial 63 (2)', 'Editorial Vol. 63 Núm. 1'])]

In [23]:
df.to_csv('Revista037.01.csv', index=False)